In [74]:
import csv

data = []
sentiment = []

with open('data/Sentiment Dataset.csv', encoding="utf8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 10000:
            break
        elif line_count == 0:
            line_count +=1
            continue
        else:
            data.append(row[3])
            sentiment.append(row[1])
            line_count += 1

In [75]:
import re
from nltk.corpus import stopwords

for index, text in enumerate(data):
    text = text.lower()
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','$URL$',text)
    text = re.sub('@[^\s]+','$USER$',text)
    data[index] = text

In [76]:
import nltk
import string
# nltk.download('punkt')

for index, text in enumerate(data):
    translator = text.maketrans('', '', string.punctuation)
    data[index] = text.translate(translator)

tokenized_data = []
for index, text in enumerate(data):
    tokenized_data.append(nltk.word_tokenize(text))
    
for index, lst in enumerate(tokenized_data):
    clean_words = [word for word in lst if word not in stopwords.words('english')]
    clean_string = " ".join(clean_words)
    tokenized_data[index] = clean_string
print (tokenized_data)

['sad apl friend', 'missed new moon trailer', 'omg already 730', 'omgaga im sooo im gunna cry ive dentist since 11 suposed 2 get crown put 30mins', 'think mi bf cheating tt', 'worry much', 'juuuuuuuuuuuuuuuuussssst chillin', 'sunny work tomorrow tv tonight', 'handed uniform today miss already', 'hmmmm wonder number USER', 'must think positive', 'thanks haters face day 112102', 'weekend sucked far', 'jb isnt showing australia', 'ok thats win', 'lt way feel right', 'awhhe man im completely useless rt funny twitter URL', 'feeling strangely fine im gon na go listen semisonic celebrate', 'huge roll thunder nowso scary', 'cut beard growing well year im gon na start USER happy meantime', 'sad iran', 'wompppp wompp', 'youre one see cause one else following youre pretty awesome', 'ltsad level 3 writing massive blog tweet myspace comp shut lost lays fetal position', 'headed hospitol pull golf tourny 3rd place think reripped something yeah', 'boring whats wrong please tell', 'cant bothered wish c

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer

Tweets_mix = []

for i, row in enumerate(tokenized_data):
    all_row = ""
    for sent in row:
        all_row += sent
    Tweets_mix.append(all_row)

tfidf_model = TfidfVectorizer(max_df=0.9, max_features=1000,
                              min_df=0.1, stop_words='english',
                              use_idf=True, tokenizer=None, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(Tweets_mix) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \

      " synoposes and " + str(tfidf_matrix.shape[1]) + " terms.")

In total, there are 9999 synoposes and 1 terms.


In [71]:
from sklearn.cluster import KMeans

num_clusters = 2
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [81]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
 
accuracy = accuracy_score(list(map(int, sentiment)), clusters)
precision, recall, f1_score, _ = precision_recall_fscore_support(list(map(int, sentiment)), clusters, average='binary')
 
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

Accuracy:  0.6364636463646365
Precision:  0.7222222222222222
Recall:  0.21423453546692142
F1 score:  0.33044759624240194
